# Finalize raw data

In [149]:
path = ('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/')

import os
os.chdir(path)

## Read in all years of data

In [150]:
import pandas as pd

df_2005_2006 = pd.read_csv('df_2005_2006_ext.csv')
df_2007_2008 = pd.read_csv('df_2007_2008_ext.csv')
df_2009_2010 = pd.read_csv('df_2009_2010_ext.csv')
df_2011_2012 = pd.read_csv('df_2011_2012_ext.csv')
df_2013_2014 = pd.read_csv('df_2013_2014_ext.csv')
df_2015_2016 = pd.read_csv('df_2015_2016_ext.csv')


In [151]:
# insert col for year
df_2005_2006['year'] = '2005-2006'
df_2007_2008['year'] = '2007-2008'
df_2009_2010['year'] = '2009-2010'
df_2011_2012['year'] = '2011-2012'
df_2013_2014['year'] = '2013-2014'
df_2015_2016['year'] = '2015-2016'

In [152]:
from functools import reduce

dfs = [df_2005_2006, df_2007_2008, df_2009_2010, df_2011_2012, df_2013_2014, df_2015_2016]

df_raw_ext = reduce(lambda  left,right: pd.concat([left,right], axis=0, ignore_index=True), dfs)

In [153]:
df_raw_ext.shape

(31357, 123)

In [154]:
df_raw_ext.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_raw_ext.csv', index=False)

## Fix columns

In [155]:
import numpy as np
# sexual orientation 6 and 5 are same category across years
df_raw_ext['sexual_orientation_M'].replace({6:5}, inplace=True)
df_raw_ext['sexual_orientation_F'].replace({6:5}, inplace=True)

# fill male NA with female answers
df_raw_ext['sexual_orientation_M'] = df_raw_ext.apply(
    lambda row: row['sexual_orientation_F'] if np.isnan(row['sexual_orientation_M']) else row['sexual_orientation_M'],
    axis=1)

# lifetime partners
# fill male NA with female answers
df_raw_ext['lifetime_male_partners'] = df_raw_ext.apply(
    lambda row: row['lifetime_female_partners'] if np.isnan(row['lifetime_male_partners']) else row['lifetime_male_partners'],
    axis=1)


# fill in doc_diabtetes with pre-diabetes if they are missing
df_raw_ext['doc_diabetes'] = df_raw_ext.apply(
    lambda row: row['doc_prediabetes'] if np.isnan(row['doc_diabetes']) else row['doc_diabetes'],
    axis=1)

# boarderline diabetes into pos
df_raw_ext['doc_diabetes'].replace({3:1}, inplace=True)


# HIV inconclusive to NaN
df_raw_ext['HIV'].replace({3:np.nan}, inplace=True)
# same for herpes
df_raw_ext['herpes_2'].replace({3:np.nan}, inplace=True)

# activity 3 goes to 2
df_raw_ext['vigorous_activity'].replace({3:2}, inplace=True)
df_raw_ext['moderate_activity'].replace({3:2}, inplace=True)


# total cholesterol is better than these two, drop female ortientation lifetime partners, and prediabetes
df_raw_ext.drop(['HDL','LDL', 'sexual_orientation_F', 'lifetime_female_partners', 'doc_prediabetes'], axis=1, inplace=True)

df_raw_ext.rename(columns={"sexual_orientation_M": "sexual_orientation", "lifetime_male_partners": "lifetime_partners"}, inplace=True)


## Replace all 7='refused' and 9='don't know' with NaN

In [156]:
cols = df_raw_ext.columns
for i in cols:
    print(i)

SEQN
depressed
broken_hip
doc_osteoporosis
race_ethnicity
edu_level
#_ppl_household
age
gender
annual_HI
marital_status
caffeine
special_diet
protein
carb
sugar
fiber
fat
diet_chol
VitB6
folate
VitB12
magnesium
iron
zinc
selenium
water
doc_diabetes
how_healthy_diet
used_CMH
gen_health_cond
place_healthcare
healthcare_past_yr
health_insurance
#_rooms
home_ownership
mom_smoked_preg
run_out_food
food_didnt_last
afford_balanced_meals
adults_skip_meals
relied_cheap_food
food_security_cat
food_stamps
WIC
doc_asthma
doc_overweight
doc_arthritis
doc_CHF
doc_CHD
doc_heart_attack
doc_stroke
doc_chronic_bronchitis
doc_liver_condition
doc_thyroid_problem
doc_cancer
anemia
cancer_type
doc_kidney
work_type
hrs_worked_week
embarassed_mouth
lab_B12
vigorous_activity
moderate_activity
hours_TV
hours_computer
limit_work
PME_limit
manage_$_difficult
doc_sleeping_disorder
hrs_sleep
smoker
smoked_5days
sexual_orientation
lifetime_partners
alcoholic
consider_weight
tried_lose_weight
triglyceride
tot_cholest

In [157]:
import numpy as np
cont = ['edu_level', '#_ppl_household', 'age', 'annual_HI', 'caffeine', 'protein', 'carb', 
        'sugar', 'fiber', 'fat', 'diet_chol', 'VitB6', 'folate', 'VitB12', 'magnesium', 'iron', 'zinc', 
        'selenium', 'water', 'healthcare_past_yr', '#_rooms', 'hrs_worked_week', 'lab_B12', 'hours_TV',
        'hours_computer', 'hrs_sleep', 'lifetime_partners', 'triglyceride', 'tot_cholesterol',
        'glycohemoglobin', 'rbc_folate', 'serum_folate', 'fasting_glucose', 'lab_VitD', 'systolic_BP', 
        'diastolic_BP','pulse', 'BMI', 'waist_C', 'VitD', '#meals_fast_food', 'monthly_income', 
        'min_sedetary', 'serum_selenium', 'serum_zinc', 'serum_creatinine',
        'lab_VitB12', 'bone_mineral_density']

In [158]:
# find the difference in columns 
def Diff(l1, l2): 
    return (list(set(l1) - set(l2))) 

non_cont = Diff(cols, cont)

In [159]:
import numpy as np

for col in non_cont:
    df_raw_ext[col].replace([7,77,777,7777,77777], [np.nan,np.nan,np.nan,np.nan,np.nan], inplace=True)
    df_raw_ext[col].replace([9,99,999,9999,99999], [np.nan,np.nan,np.nan,np.nan,np.nan], inplace=True)
    
    
# annual_HI - 99 77, min_sedetary - 7777
col_2 = ['annual_HI', 'min_sedetary', 'healthcare_past_yr', '#_rooms', 'hrs_worked_week','hours_TV',
        'hours_computer', 'hrs_sleep']

for col in col_2:
    df_raw_ext[col].replace([77,777,7777,77777], [np.nan,np.nan,np.nan,np.nan], inplace=True)
    df_raw_ext[col].replace([99,999,9999,99999], [np.nan,np.nan,np.nan,np.nan], inplace=True)

In [160]:
df_raw_ext.head()

,SEQN,depressed,broken_hip,doc_osteoporosis,race_ethnicity,edu_level,#_ppl_household,age,gender,annual_HI,...,serum_zinc,serum_creatinine,lab_VitB12,bone_mineral_density,difficult_hearing,difficult_seeing,difficult_walking,difficult_errands,doc_COPD,activity_week
0,31131.0,0.0,2.0,2.0,4.0,4.0,4.0,44.0,2.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31132.0,0.0,2.0,2.0,3.0,5.0,2.0,70.0,1.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31134.0,0.0,2.0,2.0,3.0,3.0,2.0,73.0,1.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31139.0,0.0,NaN,NaN,2.0,NaN,3.0,18.0,2.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31143.0,0.0,NaN,NaN,3.0,NaN,3.0,19.0,1.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Replace all 1=yes, 2=no with 1=True, 0=False

In [161]:
binary = df_raw_ext.columns[df_raw_ext.nunique()<=2]

categorical = Diff(non_cont, binary)

In [162]:
# all T/F columns
# replace 2 with 0
for col in binary:
    # should I replace 'indeterminant'/'boarderline' with 0 or NaN?
    df_raw_ext[col].replace([2,3], [0,0], inplace=True)


In [163]:
import numpy as np

# outliers in lifetime partners
df_raw_ext['lifetime_partners'] = np.where(df_raw_ext['lifetime_partners']>100, np.nan, df_raw_ext['lifetime_partners'])

# 'annual_HI' - 1-10 $5,000 increments, 11-$75,000 (only in 2004) 12-Over $20,000, 
# 13-Under $20,000, 14-$75,000 to $99,999, 15-$100,000 and Over
# drop 12 and 13, make 14-->12, 15-->13
df_raw_ext['annual_HI'] = np.where(df_raw_ext['annual_HI']==12, np.nan, df_raw_ext['annual_HI'])
df_raw_ext['annual_HI'] = np.where(df_raw_ext['annual_HI']==13, np.nan, df_raw_ext['annual_HI'])
df_raw_ext['annual_HI'].replace({14:12, 15:13}, inplace=True)

# outliers in caffeine
df_raw_ext['caffeine'] = np.where(df_raw_ext['caffeine']>500, np.nan, df_raw_ext['caffeine'])

#df_raw['triglyceride'] = np.where(df_raw['triglyceride']>400, np.nan, df_raw['triglyceride'])

df_raw_ext['#meals_fast_food'] = np.where(df_raw_ext['#meals_fast_food']<1,0, df_raw_ext['#meals_fast_food'])
df_raw_ext['#meals_fast_food'] = np.where(df_raw_ext['#meals_fast_food']==9999,np.nan, df_raw_ext['#meals_fast_food'])
df_raw_ext['#meals_fast_food'] = np.where(df_raw_ext['#meals_fast_food']==5555,np.nan, df_raw_ext['#meals_fast_food'])

In [165]:
df_raw_ext

,SEQN,depressed,broken_hip,doc_osteoporosis,race_ethnicity,edu_level,#_ppl_household,age,gender,annual_HI,...,serum_zinc,serum_creatinine,lab_VitB12,bone_mineral_density,difficult_hearing,difficult_seeing,difficult_walking,difficult_errands,doc_COPD,activity_week
0,31131.0,0.0,0.0,0.0,4.0,4.0,4.0,44.0,0.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31132.0,0.0,0.0,0.0,3.0,5.0,2.0,70.0,1.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31134.0,0.0,0.0,0.0,3.0,3.0,2.0,73.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31139.0,0.0,NaN,NaN,2.0,NaN,3.0,18.0,0.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31143.0,0.0,NaN,NaN,3.0,NaN,3.0,19.0,1.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31352,93691.0,0.0,NaN,NaN,5.0,5.0,7.0,25.0,1.0,10.0,...,68.5,NaN,NaN,0.733,0.0,0.0,0.0,0.0,0.0,NaN
31353,93695.0,0.0,NaN,NaN,3.0,3.0,1.0,76.0,0.0,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,NaN
31354,93696.0,0.0,NaN,NaN,3.0,5.0,3.0,26.0,1.0,9.0,...,NaN,NaN,NaN,1.226,0.0,0.0,0.0,0.0,0.0,NaN
31355,93697.0,0.0,NaN,NaN,3.0,4.0,1.0,80.0,0.0,7.0,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,NaN


In [166]:
# bring year and target col to the beginning of df
year = df_raw_ext.pop('year')
df_raw_ext.insert(1, 'year', year)

dep = df_raw_ext.pop('depressed')
df_raw_ext.insert(2, 'depressed', dep)

In [167]:
df_raw_ext.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_raw_ext.csv', index=False)

In [168]:
print(str(cont)[1:-1])

'edu_level', '#_ppl_household', 'age', 'annual_HI', 'caffeine', 'protein', 'carb', 'sugar', 'fiber', 'fat', 'diet_chol', 'VitB6', 'folate', 'VitB12', 'magnesium', 'iron', 'zinc', 'selenium', 'water', 'healthcare_past_yr', '#_rooms', 'hrs_worked_week', 'lab_B12', 'hours_TV', 'hours_computer', 'hrs_sleep', 'lifetime_partners', 'triglyceride', 'tot_cholesterol', 'glycohemoglobin', 'rbc_folate', 'serum_folate', 'fasting_glucose', 'lab_VitD', 'systolic_BP', 'diastolic_BP', 'pulse', 'BMI', 'waist_C', 'VitD', '#meals_fast_food', 'monthly_income', 'min_sedetary', 'serum_selenium', 'serum_zinc', 'serum_creatinine', 'lab_VitB12', 'bone_mineral_density'


In [146]:
print(str(non_cont)[1:-1])

'manage_$_difficult', 'health_insurance', 'gender', 'gen_health_cond', 'depressed', 'special_diet', 'alcoholic', 'limit_work', 'consider_weight', 'PME_limit', 'doc_CHD', 'food_didnt_last', 'smoker', 'anemia', 'how_healthy_diet', 'difficult_walking', 'doc_cancer', 'doc_HPV', 'nasal_congestion', 'doc_liver_condition', 'food_stamps', 'place_healthcare', 'HIV', 'cancer_type', 'work_type', 'embarassed_mouth', 'doc_thyroid_problem', 'doc_sleeping_disorder', 'activity_week', 'prob_tatse', 'doc_diabetes', 'adults_skip_meals', 'race_ethnicity', 'prob_smell', 'year', 'difficult_hearing', 'doc_asthma', 'doc_overweight', 'herpes_2', 'doc_arthritis', 'days_60min_active', 'home_ownership', 'broken_hip', 'doc_heart_attack', 'doc_osteoporosis', 'SEQN', 'food_security_cat', 'tried_lose_weight', 'relied_cheap_food', 'savings_5000', 'doc_chronic_bronchitis', 'sexual_orientation', 'difficult_remembering', 'marital_status', 'doc_COPD', 'afford_balanced_meals', 'mom_smoked_preg', 'vigorous_activity', 'doc_C

In [144]:
binary

Index(['depressed', 'broken_hip', 'doc_osteoporosis', 'gender', 'special_diet',
       'doc_diabetes', 'used_CMH', 'health_insurance', 'mom_smoked_preg',
       'adults_skip_meals', 'food_stamps', 'WIC', 'doc_asthma',
       'doc_overweight', 'doc_arthritis', 'doc_CHF', 'doc_CHD',
       'doc_heart_attack', 'doc_stroke', 'doc_chronic_bronchitis',
       'doc_liver_condition', 'doc_thyroid_problem', 'doc_cancer', 'anemia',
       'doc_kidney', 'vigorous_activity', 'moderate_activity', 'limit_work',
       'PME_limit', 'doc_sleeping_disorder', 'smoker', 'smoked_5days',
       'alcoholic', 'tried_lose_weight', 'herpes_2', 'HIV', 'savings_5000',
       'days_60min_active', 'doc_HPV', 'difficult_remembering', 'prob_smell',
       'prob_tatse', 'persistent_cold', 'nasal_congestion',
       'difficult_hearing', 'difficult_seeing', 'difficult_walking',
       'difficult_errands', 'doc_COPD', 'activity_week'],
      dtype='object')

In [148]:
print(str(categorical)[1:-1])

'manage_$_difficult', 'gen_health_cond', 'consider_weight', 'food_didnt_last', 'how_healthy_diet', 'place_healthcare', 'cancer_type', 'work_type', 'embarassed_mouth', 'race_ethnicity', 'year', 'home_ownership', 'food_security_cat', 'relied_cheap_food', 'marital_status', 'afford_balanced_meals', 'run_out_food', 'SEQN', 'oral_rec', 'sexual_orientation'
